In [ ]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

In [ ]:
def split_list(input: str, delimeter: str):
    return [int(x) for x in input.split(delimeter)]


def get_rules_and_updates(data):
    rules = []
    updates = []
    for line in data.split("\n"):
        if "|" in line:
            rules.append(split_list(line, "|"))
        elif len(line) > 0:
            updates.append(split_list(line, ","))
    return rules, updates

In [ ]:
def get_dict_x_lower_than(rules):
    result_dict = {}
    for l, r in rules:
        if l in result_dict:
            result_dict[l].append(r)
        else:
            result_dict[l] = [r]
    return result_dict

In [ ]:
def check_update(update: list, x_lower_than: dict):
    for i, v in enumerate(update):
        following_pages = update[i + 1 :]
        if len(set(following_pages) - set(x_lower_than.get(v, []))) > 0:
            return False
    return True

# Part A

In [ ]:
def part_a(data: str) -> str:
    rules, updates = get_rules_and_updates(data)
    x_lower_than = get_dict_x_lower_than(rules)
    x_greater_than = get_dict_x_greater_than(rules)
    results = 0
    for update in updates:
        if check_update(update, x_lower_than):
            results += update[(len(update) - 1) // 2]
    return str(results)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
from functools import cmp_to_key

In [ ]:
def part_b(data: str) -> str:
    result = 0
    rules, updates = get_rules_and_updates(data)
    x_lower_than = get_dict_x_lower_than(rules)
    x_greater_than = get_dict_x_greater_than(rules)

    def compare(x, y):
        if x in x_lower_than:
            if y in x_lower_than[x]:
                return -1
            else:
                return 0
        elif y in x_lower_than:
            if x in x_lower_than[y]:
                return 1
            else:
                return 0
        return 0

    for update in updates:
        if not check_update(update, x_lower_than):
            sorted_list = sorted(update, key=cmp_to_key(compare))
            result += sorted_list[(len(update) - 1) // 2]

    return str(result)

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_b="123")

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_b != "":
        print(
            f"Example {example_index} part b: {part_b(example.input_data)} (expected {example.answer_b})"
        )
        assert part_b(str(example.input_data)) == example.answer_b
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)